In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
class NN:
    def __init__(self, hidden_layers):
        hidden_layers.insert(0,77)
        hidden_layers.append(1)
        self.layers=hidden_layers
        self.L=len(hidden_layers)
        #self.beta={}
        #self.gamma={}
        self.W={}
        self.b={}
        self.keepprob={}
        self.initialize_parameters()
     
    def placeholders(self):
        X=tf.placeholder(tf.float32, shape=(None, 77), name="X")
        Y=tf.placeholder(tf.float32, shape=(None, 1), name="Y")
        return X,Y
    
    def initialize_parameters(self):
        for i in range(1, self.L):
            self.W[i]=tf.Variable(tf.random.normal(shape=(self.layers[i],self.layers[i-1])))
            self.b[i]=tf.Variable(tf.random.normal(shape=(self.layers[i],1)))
            self.keepprob[i]=tf.Variable(0.8)
            #self.beta[i]=tf.Variable(tf.random.normal(shape=(1,1)))
            #self.gamma[i]=tf.Variable(tf.random.normal(shape=(1,1)))
            
    def forward_pass(self,A, mode="train"):
        for i in range(1, self.L):
            if mode=="train":
                A = tf.nn.dropout(A, self.keepprob[i]) 
            
            Z = tf.matmul(A,tf.transpose(self.W[i]))+tf.transpose(self.b[i])
            
            if i!=self.L-1:
                A=tf.nn.sigmoid(Z)
            else:
                A=tf.nn.sigmoid(Z)
        return A
    
    def compute_cost(self,Yhat,Y):
        cost=tf.sqrt(tf.reduce_mean(tf.math.squared_difference(Yhat, Y)))
        return cost
    
    def evaluate_rmse(self, predicted, Y):
        rmse=tf.sqrt(tf.reduce_mean(tf.math.squared_difference(predicted, Y))) #root mean square error cost function
        return rmse
    
    def train(self, X_train,Y_train, learning_rate, iterations, sess):
        X,Y=self.placeholders()
        Yhat=self.forward_pass(X)
        cost=self.compute_cost(Yhat, Y)
        
        global_step = tf.Variable(0, trainable=False)
        decayed_lr = tf.train.exponential_decay(learning_rate, global_step, 1000, 0.5, staircase=True)
        trainer = tf.train.AdamOptimizer(decayed_lr, epsilon=1e-10).minimize(cost)
        init=tf.global_variables_initializer()
        sess.run(init)
        for i in range(iterations):
            _ ,temp_cost=sess.run([trainer, cost], feed_dict={X:X_train, Y:Y_train})
            if i%100==0:
                print("Cost on iteration {} :".format(i), temp_cost)
            
    def predict(self, X):
        X=tf.convert_to_tensor(X, dtype=tf.float32)
        prediction=self.forward_pass(X, "test")
        return prediction
                             

In [3]:
"""
#первый датасет Доры
X_train = pd.read_csv('nonorm_negignore_knn3/x_knn_train_negative.csv', header=0).values[:, 1:]
Y_train = pd.read_csv('nonorm_negignore_knn3/y_knn_train_negative.csv', header=0).values[:, 1:]
X_test = pd.read_csv('nonorm_negignore_knn3/x_knn_test_negative.csv', header=0).values[:, 1:]
Y_test = pd.read_csv('nonorm_negignore_knn3/y_knn_test_negative.csv', header=0).values[:, 1:]

#второй датасет Доры
X_train = pd.read_csv('norm_negignore_knn3/x_knn_train_negative_normalized.csv', header=0).values[:, 1:]
Y_train = pd.read_csv('norm_negignore_knn3/y_knn_train_negative_normalized.csv', header=0).values[:, 1:]
X_test = pd.read_csv('norm_negignore_knn3/x_knn_test_negative_normalized.csv', header=0).values[:, 1:]
Y_test = pd.read_csv('norm_negignore_knn3/y_knn_test_negative_normalized.csv', header=0).values[:, 1:]

#третий датасет Доры
X_train = pd.read_csv('norm_negignore_knn3+mean/x_train_knn+mean.csv', header=0).values[:, 1:]
Y_train = pd.read_csv('norm_negignore_knn3+mean/y_train_knn+mean.csv', header=0).values[:, 1:]
X_test = pd.read_csv('norm_negignore_knn3+mean/x_test_knn+mean.csv', header=0).values[:, 1:]
Y_test = pd.read_csv('norm_negignore_knn3+mean/y_test_knn+mean.csv', header=0).values[:, 1:]


#интерполяция + knn
X_train = pd.read_csv('datasets/norm_negignore_interpol_knn3/x_train_knn+interpol.csv', header=0).values[:, 1:]
Y_train = pd.read_csv('datasets/norm_negignore_interpol_knn3/y_train_knn+interpol.csv', header=0).values[:, 1:]
X_test = pd.read_csv('datasets/norm_negignore_interpol_knn3/x_test_knn+interpol.csv', header=0).values[:, 1:]
Y_test = pd.read_csv('datasets/norm_negignore_interpol_knn3/y_test_knn+interpol.csv', header=0).values[:, 1:]

"""
X_train = pd.read_csv('datasets/iterative_imputer/x_train_iter+knn+time.csv', header=0).values[:, 3:]
Y_train = pd.read_csv('datasets/iterative_imputer/y_train_iter+knn+time.csv', header=0).values[:, 1:]
X_test = pd.read_csv('datasets/iterative_imputer/x_test_iter+knn+time.csv', header=0).values[:, 3:]
Y_test = pd.read_csv('datasets/iterative_imputer/y_test_iter+knn+time.csv', header=0).values[:, 1:]


print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(2610, 77)
(2610, 1)
(1255, 77)
(1255, 1)


In [4]:
hidden_layers=[128, 320, 320, 50, 20, 1] #кол-во элементов это кол-во hidden layers, значение элемента это кол-во neurons в этом layer
learning_rate=0.001
iterations=10000
dropout=0.9 #probability to keep nodes

In [5]:
sess=tf.InteractiveSession()

network=NN(hidden_layers) 

In [6]:
saver = tf.train.Saver()

#чтобы вытащить модель из сохранения
#saver.restore(sess, 'prohack_tf_savedmodel/saved_variable')


In [8]:
network.train(X_train, Y_train, learning_rate, iterations, sess)

Cost on iteration 0 : 0.43835688
Cost on iteration 100 : 0.41336742
Cost on iteration 200 : 0.38867325
Cost on iteration 300 : 0.36430043
Cost on iteration 400 : 0.34043062
Cost on iteration 500 : 0.31723723
Cost on iteration 600 : 0.29486638
Cost on iteration 700 : 0.273438
Cost on iteration 800 : 0.25304356
Cost on iteration 900 : 0.23374729
Cost on iteration 1000 : 0.21559078
Cost on iteration 1100 : 0.19859105
Cost on iteration 1200 : 0.1827516
Cost on iteration 1300 : 0.16806191
Cost on iteration 1400 : 0.15450028
Cost on iteration 1500 : 0.14203764
Cost on iteration 1600 : 0.13064244
Cost on iteration 1700 : 0.12027755
Cost on iteration 1800 : 0.11090664
Cost on iteration 1900 : 0.10249087
Cost on iteration 2000 : 0.094990164
Cost on iteration 2100 : 0.08836211
Cost on iteration 2200 : 0.082561255
Cost on iteration 2300 : 0.07753728
Cost on iteration 2400 : 0.073234774
Cost on iteration 2500 : 0.0695931
Cost on iteration 2600 : 0.06654679
Cost on iteration 2700 : 0.06402782
Cost 

In [9]:
predictions_train=network.predict(X_train)
error_train=network.evaluate_rmse(predictions_train,Y_train).eval()

predictions_test=network.predict(X_test)
error_test=network.evaluate_rmse(predictions_test,Y_test).eval()

predictions_test=predictions_test.eval()
    
print("Average error on train set: ", error_train)
print("Average error on test set: ", error_test)

print("***********************")
print("predictions: ", predictions_test)

Average error on train set:  0.044448044
Average error on test set:  0.069658816
***********************
predictions:  [[0.06695151]
 [0.06689611]
 [0.06684345]
 ...
 [0.09936765]
 [0.09936765]
 [0.09936765]]


In [ ]:

#чтобы сохранить модель
#saver.save(sess, 'saved models/addfolder/addfilename')

In [ ]:
#pd.DataFrame(array of predictions).to_csv("file path", header=["y"])

In [ ]:
sess.close()